In [1]:
#!/usr/local/bin/python3

import pandas as pd
import numpy as np
from math import isnan
import bisect
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import MinMaxScaler
import skbio.stats.distance 
from scipy.spatial.distance import squareform

##### read in additional metadata from yancong that includes dysbiosis measure ###### 
hmp2_extra_meta = pd.read_csv("/Users/tobynbranck/Documents/Effect_size/data/HMP2/HMP2_MGX_metadata_09282018.csv", sep=',', header=0, index_col=0)
hmp2_extra_meta = hmp2_extra_meta[['consent_age','antibiotic','diagnosis2']]

hmp2_metadata = pd.read_csv("/Users/tobynbranck/Documents/Effect_size/data/HMP2/hmp2_metadata.csv", sep=',', header=0, index_col=0)
hmp2_metadata = hmp2_metadata.loc[hmp2_metadata['data_type'] == 'metagenomics']

hmp2_meta = hmp2_metadata.iloc[:, 70:91]
hmp2_meta = hmp2_meta.drop(['1) In the past 2 weeks, have you received any of the following medications'],axis=1)
disease = hmp2_metadata['diagnosis'].tolist()

/opt/anaconda3/lib/python3.7/site-packages/skbio/util/_testing.py:16: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as pdt
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3186: DtypeWarning: Columns (12,13,14,40,46,55,65,66,67,68,177,294,298,299,300,301,304,305,306,307,308,309,310,318,320,328,330,346,348,350,352,354,359,361,363,366,373,377,381,383,418,419,432,433,434,435,436,438,439,440,441,442,444,445,446,447,448,450,451,452,453,454) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
tea = hmp2_metadata['Tea or coffee no sugar and no sugar replacement']
hmp2_meta.insert(1, "Tea and coffee", tea)

In [3]:
hmp2_meta

,"Soft drinks, tea or coffee with sugar (corn syrup, maple syrup, cane sugar, etc)",Tea and coffee,"Diet soft drinks, tea or coffee with sugar (Stevia, Equal, Splenda etc)","Fruit juice (orange, apple, cranberry, prune etc.)",Water,"Alcohol (beer, brandy, spirits, hard liquor, wine, aperitif, etc.)","Yogurt or other foods containing active bacterial cultures (kefir, sauerkraut)","Dairy (milk, cream, ice cream, cheese, cream cheese)",Probiotic,"Fruits (no juice) (Apples, raisins, bananas, oranges, strawberries, blueberries",...,"Beans (tofu, soy, soy burgers, lentils, Mexican beans, lima beans etc)","Whole grains (wheat, oats, brown rice, rye, quinoa, wheat bread, wheat pasta)","Starch (white rice, bread, pizza, potatoes, yams, cereals, pancakes, etc.)",Eggs,"Processed meat (other red or white meat such as lunch meat, ham, salami, bologna","Red meat (beef, hamburger, pork, lamb)","White meat (chicken, turkey, etc.)","Shellfish (shrimp, lobster, scallops, etc.)","Fish (fish nuggets, breaded fish, fish cakes, salmon, tuna, etc.)","Sweets (pies, jam, chocolate, cake, cookies, etc.)"
Project,,,,,,,,,,,,,,,,,,,,,
G79889,"Yesterday, 1 to 2 times","No, I did not consume these products in the la...","No, I did not consume these products in the la...",Within the past 4 to 7 days,"Yesterday, 1 to 2 times","No, I did not consume these products in the la...",Within the past 2 to 3 days,"Yesterday, 1 to 2 times","No, I did not consume these products in the la...",Within the past 2 to 3 days,...,Within the past 2 to 3 days,"Yesterday, 1 to 2 times","Yesterday, 1 to 2 times",Within the past 2 to 3 days,"No, I did not consume these products in the la...","No, I did not consume these products in the la...","No, I did not consume these products in the la...","No, I did not consume these products in the la...",Within the past 2 to 3 days,Within the past 2 to 3 days
G79894,"Yesterday, 1 to 2 times",Within the past 4 to 7 days,"No, I did not consume these products in the la...",Within the past 4 to 7 days,"Yesterday, 1 to 2 times","No, I did not consume these products in the la...",Within the past 2 to 3 days,"Yesterday, 1 to 2 times","No, I did not consume these products in the la...",Within the past 2 to 3 days,...,Within the past 4 to 7 days,"Yesterday, 1 to 2 times","Yesterday, 1 to 2 times",Within the past 4 to 7 days,"No, I did not consume these products in the la...","No, I did not consume these products in the la...","No, I did not consume these products in the la...","No, I did not consume these products in the la...",Within the past 4 to 7 days,Within the past 2 to 3 days
G79903,"Yesterday, 1 to 2 times",Within the past 4 to 7 days,"No, I did not consume these products in the la...",Within the past 2 to 3 days,"Yesterday, 1 to 2 times","No, I did not consume these products in the la...","Yesterday, 1 to 2 times","Yesterday, 3 or more times","Yesterday, 1 to 2 times","Yesterday, 1 to 2 times",...,"Yesterday, 1 to 2 times","Yesterday, 1 to 2 times","Yesterday, 1 to 2 times",Within the past 4 to 7 days,"No, I did not consume these products in the la...","No, I did not consume these products in the la...","No, I did not consume these products in the la...",Within the past 4 to 7 days,Within the past 4 to 7 days,Within the past 2 to 3 days
G79913,"Yesterday, 1 to 2 times","Yesterday, 1 to 2 times","No, I did not consume these products in the la...",Within the past 2 to 3 days,"Yesterday, 1 to 2 times","No, I did not consume these products in the la...","Yesterday, 1 to 2 times","Yesterday, 1 to 2 times","No, I did not consume these products in the la...","Yesterday, 1 to 2 times",...,"Yesterday, 1 to 2 times","Yesterday, 1 to 2 times","Yesterday, 1 to 2 times",Within the past 2 to 3 days,"No, I did not consume these products in the la...","No, I did not consume these products in the la...","No, I did not consume these products in the la...",Within the past 4 to 7 days,Within the past 4 to 7 days,"Yesterday, 1 to 2 times"
G79926,"Yesterday, 1 t

In [4]:
#coding diet normalized by serving's per day
column_codes = {"Soft drinks, tea or coffee with sugar (corn syrup, maple syrup, cane sugar, etc)": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Diet soft drinks, tea or coffee with sugar (Stevia, Equal, Splenda etc)":  {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3}, 
                "Fruit juice (orange, apple, cranberry, prune etc.)": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Water": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Alcohol (beer, brandy, spirits, hard liquor, wine, aperitif, etc.)": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Yogurt or other foods containing active bacterial cultures (kefir, sauerkraut)": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Dairy (milk, cream, ice cream, cheese, cream cheese)": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Probiotic": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Fruits (no juice) (Apples, raisins, bananas, oranges, strawberries, blueberries": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Vegetables (salad, tomatoes, onions, greens, carrots, peppers, green beans, etc)": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Beans (tofu, soy, soy burgers, lentils, Mexican beans, lima beans etc)": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Whole grains (wheat, oats, brown rice, rye, quinoa, wheat bread, wheat pasta)": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Starch (white rice, bread, pizza, potatoes, yams, cereals, pancakes, etc.)": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Eggs": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Processed meat (other red or white meat such as lunch meat, ham, salami, bologna": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Red meat (beef, hamburger, pork, lamb)": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "White meat (chicken, turkey, etc.)": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Shellfish (shrimp, lobster, scallops, etc.)": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Fish (fish nuggets, breaded fish, fish cakes, salmon, tuna, etc.)": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Sweets (pies, jam, chocolate, cake, cookies, etc.)": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3},
                "Tea and coffee": {"No, I did not consume these products in the last 7 days": 0, "Within the past 4 to 7 days": .143, "Within the past 2 to 3 days": .286, "Yesterday, 1 to 2 times": 1.5, "Yesterday, 3 or more times": 3}}

hmp2_meta.replace(column_codes, inplace=True)
hmp2_meta['IDs'] = hmp2_metadata['External ID']
hmp2_meta['Participant_ID'] = hmp2_metadata['Participant ID']
#binning
bins = {0: (0,7), 1: (0,7), 2: (0,7), 3: (0,7), 4: (0,7), 5: (0,7), 
    6: (0,7), 7: (0,7), 8: (8,15), 9: (8,15), 10: (8,15), 11: (8,15),
    12: (8,15), 13: (8,15), 14: (8,15), 15: (8,15), 16: (16,24),
    17: (16,24), 18: (16,24), 19: (16,24), 20: (16,24), 21: (16,24),
    22: (16,24), 23: (16,24), 24: (16,24), 25: (25,33), 26: (25,33),
    27: (25,33), 28: (25,33), 29: (25,33), 30: (25,33), 31: (25,33),
    32: (25,33), 33: (25,33), 34: (34,42), 35: (34,42), 36: (34,42),
    37: (34,42), 38: (34,42), 39: (34,42), 40: (34,42), 41: (34,42),
    42: (34,42), 43: (43,57), 44: (43,57), 45: (43,57), 46: (43,57),
    47: (43,57), 48: (43,57), 49: (43,57), 50: (43,57), 51: (43,57),
    52: (43,57), 53: (43,57), 54: (43,57), 55: (43,57), 56: (43,57),
    47: (43,57)}
bin_list = [(0,7),(8,15),(16,24),(25,33),(34,42),(43,57)]
hmp2_meta['bin']= hmp2_metadata['week_num'].map(bins)
hmp2_meta.set_index('IDs', drop = True, inplace = True)
hmp2_meta['diagnosis'] = disease
print(hmp2_meta.shape)
hmp2_meta = hmp2_meta.dropna()
hmp2_meta.sort_index(inplace = True)
print (hmp2_meta.shape)


(1638, 24)
(1492, 24)


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5591: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  key,


In [5]:
hmp2_meta
#remove two columns in dataframe
hmp2_meta.drop(['Probiotic', 'Water'], axis=1, inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
####incorporate additional metadata by merging two dataframes
hmp2_meta = pd.merge(hmp2_meta, hmp2_extra_meta, left_index=True, right_index=True)

#selecting for age
hmp2_meta = hmp2_meta.loc[hmp2_meta['consent_age'] >= 18]

#make dict from external ID (ID that matches species document) and person ID (so that we can make a person ID column in the species folder for averaging)
ID_cols_dict = dict(zip(hmp2_meta.index.values,hmp2_meta.Participant_ID))
keys_values = ID_cols_dict.items()
ID_cols_dict = {str(key): str(value) for key, value in keys_values}

#external ID mapped to bin type
ID_bin_dict = dict(zip(hmp2_meta.index.values,hmp2_meta.bin))
keys_values = ID_bin_dict.items()
ID_bin_dict = {str(key): str(value) for key, value in keys_values}
hmp2_meta.to_csv('/Users/tobynbranck/Documents/Effect_size/analysis/hmp2_meta_allPhenotypes.csv', sep=',')


In [7]:
# seperating based on disease
df_CD = hmp2_meta[hmp2_meta['diagnosis'] == 'CD']
df_UC = hmp2_meta[hmp2_meta['diagnosis'] == 'UC']
df_nonIBD = hmp2_meta[hmp2_meta['diagnosis'] == 'nonIBD']
#df_nonIBD_2 = hmp2_meta[hmp2_meta['diagnosis'] == 'nonIBD'] ####if separating based on ibd vs. nonibd
#df_IBD = hmp2_meta[hmp2_meta['diagnosis'] != 'nonIBD']
df_list = [df_CD, df_UC, df_nonIBD]
df_names = ['df_CD', 'df_UC', 'df_nonIBD']
#df_list = [df_IBD, df_nonIBD_2]
#df_names = ['df_IBD', 'df_nonIBD_2']
for idx,df in enumerate(df_list):
    df_name = df_names[idx]
    print(df)
    df.to_csv('/Users/tobynbranck/Documents/Effect_size/analysis/{}.csv'.format(df_name), sep=',')

            Soft drinks, tea or coffee with sugar (corn syrup, maple syrup, cane sugar, etc)  \
CSM5FZ3N_P                                              1.500                                  
CSM5FZ3R_P                                              1.500                                  
CSM5FZ3T_P                                              0.000                                  
CSM5FZ3V_P                                              1.500                                  
CSM5FZ3X_P                                              0.000                                  
...                                                       ...                                  
MSMAPC6C                                                1.500                                  
MSMAPC6E                                                0.000                                  
MSMAPC6G                                                0.000                                  
MSMB4LXY                                

In [8]:
####Read in species matrix####
#reading in species file
hmp_species = pd.read_csv("/Users/tobynbranck/Documents/Effect_size/data/HMP2/hmp2_species.txt", sep="\t", header=0, index_col=0)
hmp_species = hmp_species.drop(['NCBI_tax_id'], axis=1)
hmp_species.columns = hmp_species.columns.str.replace(r'_profile$', '')
hmp_species.index = hmp_species.index.str.split('s__').str[1]
hmp_species.index = hmp_species.index.str.replace(r'_', ' ', regex=True)
hmp_species_for_strict_filt = hmp_species
print (hmp_species.shape,"ORIGINAL")
##filtering for prevalence and abundance: must be at least 10^-5 in at least 25% samples
hmp_species = hmp_species.div(100)
test = (hmp_species >= 0.00001).sum(axis=1)
num_samples_thresh = hmp_species.shape[1]*0.25
rows_keep = test >= num_samples_thresh
hmp_species = hmp_species[rows_keep]

print ("HMP", hmp_species.head)

print (hmp_species.shape,"25% FILTER")
print (hmp_species.head)

#transposing hmp_species df so that can add person ID and average be person within bins
hmp_species = hmp_species.T
hmp_species['bin']= hmp_species.index.map(ID_bin_dict)
hmp_species['Participant_ID']= hmp_species.index.map(ID_cols_dict) #removes ~900 samples because diet info not available for those participants (less participants/samples in metadata file than species file)
hmp_species = hmp_species[hmp_species['Participant_ID'].notna()]
hmp_species.to_csv('/Users/tobynbranck/Documents/Effect_size/analysis/hmp_species_df.csv', sep=',')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


(578, 1638) ORIGINAL
HMP <bound method NDFrame.head of                                   PSMB4MC7  PSMB4MC5  PSMB4MC3  PSMB4MC1  \
clade_name                                                                 
Bifidobacterium adolescentis      0.000429  0.000049  0.000000  0.000383   
Bifidobacterium longum            0.000307  0.001909  0.000057  0.000704   
Collinsella aerofaciens           0.001770  0.000000  0.000000  0.000000   
Bacteroides caccae                0.016788  0.000000  0.000000  0.000000   
Bacteroides cellulosilyticus      0.000000  0.000000  0.000000  0.000000   
...                                    ...       ...       ...       ...   
Bilophila wadsworthia             0.000000  0.000000  0.000784  0.000987   
Escherichia coli                  0.000000  0.000000  0.000000  0.002336   
Haemophilus parainfluenzae        0.000121  0.000617  0.000031  0.000136   
Proteobacteria bacterium CAG 139  0.000000  0.000000  0.000000  0.000000   
Akkermansia muciniphila          

In [9]:
##filtering for prevalence and abundance: must be at least 10^-5 in at least 25% samples (note --> no longer using strict, its same as less strict filter, just keeping variables names the same)
hmp_species_strict = hmp_species_for_strict_filt.div(100)
test = (hmp_species_strict >= 0.00001).sum(axis=1)
num_samples_thresh = hmp_species_strict.shape[1]*0.25
rows_keep = test >= num_samples_thresh
hmp_species_strict = hmp_species_strict[rows_keep]
print ("HMP", hmp_species_strict.head)

print (hmp_species_strict.shape,"STRICT FILTER")
print (hmp_species_strict.head)

#transposing hmp_species df so that can add person ID and average be person within bins
hmp_species_strict = hmp_species_strict.T
hmp_species_strict['bin']= hmp_species_strict.index.map(ID_bin_dict)
hmp_species_strict['Participant_ID']= hmp_species_strict.index.map(ID_cols_dict) #removes ~900 samples because diet info not available for those participants (less participants/samples in metadata file than species file)
hmp_species_strict = hmp_species_strict[hmp_species_strict['Participant_ID'].notna()]
hmp_species_strict.to_csv('/Users/tobynbranck/Documents/Effect_size/analysis/hmp_species_df_strict.csv', sep=',')

HMP <bound method NDFrame.head of                                   PSMB4MC7  PSMB4MC5  PSMB4MC3  PSMB4MC1  \
clade_name                                                                 
Bifidobacterium adolescentis      0.000429  0.000049  0.000000  0.000383   
Bifidobacterium longum            0.000307  0.001909  0.000057  0.000704   
Collinsella aerofaciens           0.001770  0.000000  0.000000  0.000000   
Bacteroides caccae                0.016788  0.000000  0.000000  0.000000   
Bacteroides cellulosilyticus      0.000000  0.000000  0.000000  0.000000   
...                                    ...       ...       ...       ...   
Bilophila wadsworthia             0.000000  0.000000  0.000784  0.000987   
Escherichia coli                  0.000000  0.000000  0.000000  0.002336   
Haemophilus parainfluenzae        0.000121  0.000617  0.000031  0.000136   
Proteobacteria bacterium CAG 139  0.000000  0.000000  0.000000  0.000000   
Akkermansia muciniphila           0.000000  0.000000  

In [10]:
hmp_species

clade_name,Bifidobacterium adolescentis,Bifidobacterium longum,Collinsella aerofaciens,Bacteroides caccae,Bacteroides cellulosilyticus,Bacteroides dorei,Bacteroides fragilis,Bacteroides ovatus,Bacteroides stercoris,Bacteroides thetaiotaomicron,...,Veillonella parvula,Parasutterella excrementihominis,Turicimonas muris,Bilophila wadsworthia,Escherichia coli,Haemophilus parainfluenzae,Proteobacteria bacterium CAG 139,Akkermansia muciniphila,bin,Participant_ID
MSMB4LZZ,0.000065,0.000187,0.000036,0.013726,0.043526,0.155301,0.000000,0.000000,0.014830,0.028507,...,0.000237,0.000435,0.000000,0.000350,0.000000,0.000346,0.000000,0.000000,"(43, 57)",M2084
MSMB4LZX,0.001197,0.000728,0.000233,0.008507,0.014467,0.098475,0.000000,0.000000,0.004416,0.027219,...,0.000000,0.001691,0.000000,0.000462,0.000000,0.000460,0.000000,0.000000,"(43, 57)",M2084
MSMB4LZV,0.000000,0.000019,0.000234,0.003749,0.012981,0.115318,0.000000,0.000000,0.002415,0.007138,...,0.000000,0.000032,0.000000,0.001782,0.000000,0.001101,0.000000,0.000000,"(43, 57)",M2084
MSMB4LZR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043545,0.000000,0.000000,0.000085,...,0.000000,0.014878,0.000262,0.000000,0.000434,0.000000,0.001050,0.000000,"(43, 57)",M2083
MSMB4LZP,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060323,0.000000,0.000000,0.000091,...,0.000021,0.018424,0.000152,0.000000,0.000571,0.000000,0.001295,0.000000,"(43, 57)",M2083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CSM5FZ3X_P,0.000000,0.000000,0.000164,0.000000,0.000000,0.002545,0.169849,0.000000,0.001705,0.056273,...,0.000000,0.000000,0.000000,0.000000,0.018558,0.000000,0.000000,0.000000,"(0, 7)",C3002
CSM5FZ3V_P,0.000000,0.000000,0.000000,0.027069,0.000000,0.008434,0.091844,0.000776,0.000000,0.112838,...,0.000000,0.000000,0.000000,0.000000,0.000365,0.000000,0.000000,0.007645,"(0, 7)",C3001
CSM5FZ3T_P,0.000000,0.000000,0.000000,0.000000,0.000000,0.002886,0.110839,0.000000,0.000000,0.135590,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"(0, 7)",C3002
CSM5FZ3R_P,0.000077,0.000506,0.000000,0.014567,0.000102,0.000000,0.005534,0.000098,0.010125,0.086952,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"(0, 7)",C3001


In [11]:
print(len(hmp_species.Participant_ID.unique()))
print(len(df_CD.Participant_ID.unique()))
print(len(df_UC.Participant_ID.unique()))
print(len(df_nonIBD.Participant_ID.unique()))


print(hmp_species.shape)
print(df_CD.shape)
print(df_UC.shape)
print(df_nonIBD.shape)

63
28
21
14
(768, 71)
(322, 25)
(249, 25)
(197, 25)


In [12]:
hmp_species_strict

clade_name,Bifidobacterium adolescentis,Bifidobacterium longum,Collinsella aerofaciens,Bacteroides caccae,Bacteroides cellulosilyticus,Bacteroides dorei,Bacteroides fragilis,Bacteroides ovatus,Bacteroides stercoris,Bacteroides thetaiotaomicron,...,Veillonella parvula,Parasutterella excrementihominis,Turicimonas muris,Bilophila wadsworthia,Escherichia coli,Haemophilus parainfluenzae,Proteobacteria bacterium CAG 139,Akkermansia muciniphila,bin,Participant_ID
MSMB4LZZ,0.000065,0.000187,0.000036,0.013726,0.043526,0.155301,0.000000,0.000000,0.014830,0.028507,...,0.000237,0.000435,0.000000,0.000350,0.000000,0.000346,0.000000,0.000000,"(43, 57)",M2084
MSMB4LZX,0.001197,0.000728,0.000233,0.008507,0.014467,0.098475,0.000000,0.000000,0.004416,0.027219,...,0.000000,0.001691,0.000000,0.000462,0.000000,0.000460,0.000000,0.000000,"(43, 57)",M2084
MSMB4LZV,0.000000,0.000019,0.000234,0.003749,0.012981,0.115318,0.000000,0.000000,0.002415,0.007138,...,0.000000,0.000032,0.000000,0.001782,0.000000,0.001101,0.000000,0.000000,"(43, 57)",M2084
MSMB4LZR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043545,0.000000,0.000000,0.000085,...,0.000000,0.014878,0.000262,0.000000,0.000434,0.000000,0.001050,0.000000,"(43, 57)",M2083
MSMB4LZP,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060323,0.000000,0.000000,0.000091,...,0.000021,0.018424,0.000152,0.000000,0.000571,0.000000,0.001295,0.000000,"(43, 57)",M2083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CSM5FZ3X_P,0.000000,0.000000,0.000164,0.000000,0.000000,0.002545,0.169849,0.000000,0.001705,0.056273,...,0.000000,0.000000,0.000000,0.000000,0.018558,0.000000,0.000000,0.000000,"(0, 7)",C3002
CSM5FZ3V_P,0.000000,0.000000,0.000000,0.027069,0.000000,0.008434,0.091844,0.000776,0.000000,0.112838,...,0.000000,0.000000,0.000000,0.000000,0.000365,0.000000,0.000000,0.007645,"(0, 7)",C3001
CSM5FZ3T_P,0.000000,0.000000,0.000000,0.000000,0.000000,0.002886,0.110839,0.000000,0.000000,0.135590,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"(0, 7)",C3002
CSM5FZ3R_P,0.000077,0.000506,0.000000,0.014567,0.000102,0.000000,0.005534,0.000098,0.010125,0.086952,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"(0, 7)",C3001
